In [1]:
#Importando as bibliotecas
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from requests_html import HTML
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.common.exceptions import WebDriverException
from tqdm import tqdm
import sqlite3
import datetime
import os

In [2]:
#Congiruando o driver
options = Options()
options.add_argument("--headless")
options.add_argument('--disable-gpu')
options.add_argument("--log-level=3")
options.add_argument('--no-sandbox')
options.add_experimental_option('useAutomationExtension', False)
options.add_experimental_option('excludeSwitches', ['enable-logging'])

In [3]:
#Criando listas
Urls_amazon = []

Urls_amazon_more = []

Products_Links = []

Amazon_price = []
Amazon_price_2 = []

Amazon_seller = []
Amazon_seller_2 = []

Amazon_title = []
Amazon_title_2 = []

Amazon_installment_price_full = []
Amazon_installment_price_full_2 = []

Amazon_seller_more = []
Amazon_price_more = []
Amazon_title_more = []
internacional_list = []
more_offers_list = []

In [4]:
driver = webdriver.Chrome(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Data\Selenium\Selenium_101", options=options)

C:\Users\pedro\AppData\Local\Temp\ipykernel_7544\3380238038.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Data\Selenium\Selenium_101", options=options)


In [5]:
#Função para criar os links de busca
def getting_n_creating_amazon(brand):

    Database_path = r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1" + "\Data\\" + brand + "\\" + brand + ".db"

    table = brand + "_products"

    #Criando a Query
    query = "SELECT * FROM " + table

    #Entrando dentro do databse
    connection = sqlite3.connect(Database_path)

    #Criando o dataset em brando
    df = pd.read_sql_query(query, connection)

    #Passando todo o Dataframe para LowerCase
    df = df.apply(lambda x: x.astype(str).str.lower())

    #Arrumando espaços vazios
    # Arrumano os espaços vazios
    df['Product_Name'] = df['Product_Name'].str.replace(" ", "+")

    # Criando uma nova coluna no database com a formatação certa
    df['Urls'] = df['Brand'] + "+" + df['Product_Name']

    # Criando a nova coluna que são as urls de pesquisa
    df['Urls_search'] = "https://www.amazon.com.br/s?k=" + df['Urls']

    return df

In [6]:
Df_Products_raw = getting_n_creating_amazon("GoPro")
Df_Products_raw

,Brand,Product_Name,Product_Description,Urls,Urls_search
0,gopro,hero+8,câmera hero 8,gopro+hero+8,https://www.amazon.com.br/s?k=gopro+hero+8
1,gopro,hero+9,câmera hero 9,gopro+hero+9,https://www.amazon.com.br/s?k=gopro+hero+9
2,gopro,hero+10,câmera hero 10,gopro+hero+10,https://www.amazon.com.br/s?k=gopro+hero+10
3,gopro,max+360,câmera max 360,gopro+max+360,https://www.amazon.com.br/s?k=gopro+max+360


In [7]:
#Função pegando Links
def search_links(url):
    global Urls_amazon

    time.sleep(3)

    driver.get(url)
    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')
    html_obj = HTML(html=html_str)

    Links = [x for x in html_obj.links]
    products_links = [f'https://www.amazon.com.br{x}' for x in Links]

    Soup = BeautifulSoup(html_str,'html.parser')

    for link in products_links:
        Urls_amazon.append(link)

    Urls_amazon = [s for s in Urls_amazon if '/dp/' in s]
    Urls_amazon = [s for s in Urls_amazon if not '#customerReviews' in s]

In [8]:
for url in tqdm(Df_Products_raw['Urls_search']):
    search_links(url)

100%|██████████| 4/4 [00:21<00:00,  5.36s/it]


In [9]:
#Limpando as urls
def clean_link(urls, brand):
    #Criando a conexão com a tabela de exclusão 
    Database_exclusion = r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1" + "\Data\\" + brand + "\\" + brand + ".db"

    table = brand + "_exclusao"

    #Criando a Query
    query = "SELECT * FROM " + table

    connection = sqlite3.connect(Database_exclusion)

    #Criando o dataset em brando
    df_itens = pd.read_sql_query(query, connection)

    clean_urls = pd.DataFrame()

    clean_urls['Urls_Completas'] = urls
    clean_urls['Urls_limpas'] = clean_urls['Urls_Completas'].str.partition("ref")[0]

    Urls_limpas = clean_urls['Urls_limpas'].tolist()

    for word in df_itens['Words']:
         Urls_limpas = [s for s in Urls_limpas if not word in s]

    clean_urls = pd.DataFrame()

    clean_urls['Urls_finais'] = Urls_limpas

    clean_urls['ASIN'] = clean_urls['Urls_finais'].str.partition("/dp/")[2].str.partition("/")[0]

    clean_urls.drop_duplicates(subset='ASIN',inplace=True)
    clean_urls.reset_index(inplace=True, drop=True)
    return clean_urls

In [10]:
len(Urls_amazon)

194

In [31]:
Urls_amazon

['https://www.amazon.com.br/Eletronica-Digital-C%C3%A2mera-Multikids-Baby/dp/B076LD2CMK/ref=sr_1_20?keywords=motorola+mbp481&qid=1652903439&sr=8-20&ufe=app_do%3Aamzn1.fos.25548f35-0de7-44b3-b28e-0f56f3f96147',
 'https://www.amazon.com.br/Carregador-Parede-20W-Sa%C3%ADda-MC-207/dp/B09C6NYKCG/ref=sr_1_46?keywords=motorola+mbp481&qid=1652903439&sr=8-46&ufe=app_do%3Aamzn1.fos.6a09f7ec-d911-4889-ad70-de8dd83c8a74',
 'https://www.amazon.com.br/Celular-Motorola-Moto-E7-Met%C3%A1lico/dp/B08WJP1QJ7/ref=sr_1_45?keywords=motorola+mbp481&qid=1652903439&sr=8-45',
 'https://www.amazon.com.br/Dados-Micro-USB-Lifeproof-MO-SKN6461AIBI-Preto/dp/B07B41GCHY/ref=sr_1_41?keywords=motorola+mbp481&qid=1652903439&sr=8-41',
 'https://www.amazon.com.br/Smartphone-Motorola-Moto-64GB-Ros%C3%AA/dp/B09LMRZV2W/ref=sr_1_39?keywords=motorola+mbp481&qid=1652903439&sr=8-39&ufe=app_do%3Aamzn1.fos.25548f35-0de7-44b3-b28e-0f56f3f96147',
 'https://www.amazon.com.br/Carregador-Parede-Motorola-100-240V-SC-47/dp/B08D3W1ZVT/ref=

In [11]:
New_urls = clean_link(Urls_amazon, 'GoPro')

In [12]:
New_urls.shape

(43, 2)

In [13]:
def search_atributes(url):
    #Tempo para não haver o bloqueio
    time.sleep(10)

    #Entrando dentro do site com o driver
    driver.get(url)

    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')

    #Criando o Soup
    soup = BeautifulSoup(html_str, 'html.parser')

    #Fazendo o try do nome do vendedor
    try:
        seller = soup.find(id='sellerProfileTriggerId').text
        seller_2 = soup.find("a", attrs={"id":'sellerProfileTriggerId'}).text
        Amazon_seller.append(seller)
        Amazon_seller_2.append(seller_2)
    except:
        Amazon_seller.append("Erro")
        Amazon_seller_2.append("Erro")

    #Fazendo o try do preço do produto a vista
    try:
        price = soup.find(class_='a-offscreen').text
        Amazon_price.append(price)

        Div_Price = soup.find('div', attrs={"data-feature-name":"corePrice"})
        price_2 = Div_Price.find(class_='a-offscreen').text
        Amazon_price_2.append(price_2)
    except:
        Amazon_price.append("Erro")
        Amazon_price_2.append("Erro")

    #Pegando o título do produto
    try:
        title = soup.find(id='productTitle').text
        Amazon_title.append(title)
    except:
        Amazon_title.append('Erro')

    #Pegando o internacional
    try:
        soup.find('img', attrs={'data-a-hires':'https://images-na.ssl-images-amazon.com/images/G/32/foreignseller/Foreign_Seller_Badge_v2._CB403622375_.png'})
        internacional_list.append("Internacional")
    except:
        internacional_list.append("Nacional")

    #Fazendo o try para pegar o preço da parcela
    try:
        installment = soup.find(class_='best-offer-name a-text-bold').text
        Amazon_installment_price_full.append(installment)
        
        Div_installment = soup.find('div', attrs={'id':'installmentCalculator_feature_div'})
        Installment_2 = Div_installment.find(class_='best-offer-name a-text-bold').text
        Amazon_installment_price_full_2.append(Installment_2)
    except:
        Amazon_installment_price_full.append("0")
        Amazon_installment_price_full_2.append("Erro")

    #Fazendo o try para ver se tem mais ofertas
    try:
        Main_Div_More_offers = soup.find('div', attrs={"id":"olpLinkWidget_feature_div"})
        Div_More_offers = Main_Div_More_offers.find('div', attrs={'class':'a-section olp-link-widget'})
        Div_More_offers_text = Div_More_offers.find('div', attrs={'class':'olp-text-box'}).text
        more_offers_list.append(Div_More_offers_text)
    except:
        more_offers_list.append("Comparar outras 0 ofertas")


In [14]:
for url in tqdm(New_urls['Urls_finais']):
    search_atributes(url)

100%|██████████| 43/43 [08:50<00:00, 12.33s/it]


In [16]:
print(len(Amazon_title))
print(len(Amazon_price_2))
print(len(Amazon_seller_2))
print(len(more_offers_list))
print(len(Amazon_installment_price_full_2))
print(len(internacional_list))

43
43
43
43
43
43


In [18]:
def dataset_amazon(url, sellers, preco, titulo, more_url, brand):
    # Criando o DataFrame
    Dataset_amazon = pd.DataFrame()

    #
    Hoje = pd.to_datetime('today', errors='ignore').date()

    # Colocando os dados
    Dataset_amazon['URL'] = url
    
    Dataset_amazon['DATE'] = Hoje

    Dataset_amazon['MARKETPLACE'] = "AMAZON"

    

    #Arrumando a coluna de sellers
    Dataset_amazon['SELLER'] = sellers
    Dataset_amazon['SELLER'] = Dataset_amazon['SELLER'].str.replace("Erro","Amazon", regex=False)

    #Arrumando o preço 
    Dataset_amazon['PRICE'] = preco
    Dataset_amazon['PRICE'] = Dataset_amazon['PRICE'].str.replace(".","", regex=True)
    Dataset_amazon['PRICE'] = Dataset_amazon['PRICE'].str.replace("R$","", regex=False)
    Dataset_amazon['PRICE'] = Dataset_amazon['PRICE'].str.replace(",",".", regex=True)
    

    #Arrumando os valores de installment 
    Dataset_amazon['INSTALLMENT FULL'] = Amazon_installment_price_full
    Dataset_amazon['PARCEL'] = Dataset_amazon['INSTALLMENT FULL'].str.extract('(\d+)')
    Dataset_amazon['PARCEL'] = Dataset_amazon['PARCEL'].astype("int")
    Dataset_amazon['parcel_price_bruto'] = \
    Dataset_amazon['INSTALLMENT FULL'].str.partition("R$")[2].str.partition(" ")[2].str.partition(" ")[0]
    Dataset_amazon['Installment3'] = Dataset_amazon['parcel_price_bruto'].str.extract('(\d+)')
    Dataset_amazon['parcel_price_bruto'] = \
    Dataset_amazon['INSTALLMENT FULL'].str.partition("R$")[2].str.partition(" ")[2].str.partition(" ")[0].str.partition(",")[2]
    Dataset_amazon['Installment4'] = Dataset_amazon['parcel_price_bruto'].str.extract('(\d+)')
    Dataset_amazon['INSTALLMENT'] = Dataset_amazon['Installment3'] + "." + Dataset_amazon['Installment4']
    Dataset_amazon['INSTALLMENT'] = Dataset_amazon['INSTALLMENT'].astype("float")
    Dataset_amazon['INSTALLMENT'] = Dataset_amazon['INSTALLMENT'].fillna(0)
    Dataset_amazon['INSTALLMENT_PAYMENT'] = Dataset_amazon['PARCEL'] * Dataset_amazon['INSTALLMENT']


    Dataset_amazon['ID'] = Dataset_amazon['URL'].str.partition('/dp/')[2].str.partition('/')[0]
    Dataset_amazon['PRODUCT'] = titulo
    Dataset_amazon['INTERNACIONAL'] = internacional_list

    #Arrumando valores de mais sellers
    Dataset_amazon['MORE'] = more_url
    Dataset_amazon['MORE'] = Dataset_amazon['MORE'].str.partition("outras ")[2].str.partition(" ofertas")[0]
    
    Dataset_amazon['MORE'] = Dataset_amazon['MORE'].astype('int')
    

    Dataset_amazon = Dataset_amazon.drop(columns=["INSTALLMENT FULL", "Installment3", "Installment4", "parcel_price_bruto"])

    #Pegando os itens certos 
    Dataset_amazon = Dataset_amazon[Dataset_amazon["PRICE"] != "Erro"]
    Dataset_amazon['PRICE'] = Dataset_amazon['PRICE'].astype('float')


    #Colocando na ordem correta
    Dataset_amazon = Dataset_amazon[['DATE', 'URL', 'MARKETPLACE', 'SELLER', 'PRICE', 'PARCEL', 'INSTALLMENT', 'INSTALLMENT_PAYMENT', 'ID', 'PRODUCT','INTERNACIONAL', 'MORE']]

    if brand == "GoPro":
        Dataset_amazon = Dataset_amazon[Dataset_amazon['PRICE'] > 900]
    elif brand == 'Motorola':
        Dataset_amazon = Dataset_amazon[Dataset_amazon['PRICE'] > 70]
    elif brand == 'Wacom':
        Dataset_amazon = Dataset_amazon[Dataset_amazon['PRICE'] > 100]
    

    return Dataset_amazon

In [20]:
Df_final = dataset_amazon(New_urls['Urls_finais'], Amazon_seller_2, Amazon_price_2, Amazon_title, more_offers_list, 'GoPro')

In [61]:
Df_final=  Df_final.reset_index(drop=True)

In [62]:
Df_final

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL
0,2022-05-21,https://www.amazon.com.br/Gopro-Black-prova-D%...,AMAZON,EOS IMPORT,2299.00,10,229.90,2299.0,B07WSRXMS8,Gopro Hero 8 Black 4k à prova D'água C...,Internacional
1,2022-05-21,https://www.amazon.com.br/GoPro-Frontal-HyperS...,AMAZON,EOS IMPORT,3207.25,10,320.77,3207.7,B09D15SFMQ,Câmera GoPro HERO10 Black à Prova com ...,Internacional
2,2022-05-21,https://www.amazon.com.br/C%C3%A2mera-GoPro-HE...,AMAZON,Amazon,4688.20,10,468.82,4688.2,B093JKGDJZ,Câmera GoPro HERO9 Black - Kit Trip Pr...,Internacional
3,2022-05-21,https://www.amazon.com.br/C%C3%A2mera-Cart%C3%...,AMAZON,EOS IMPORT,3000.00,10,300.00,3000.0,B08R2XCM9K,Câmera GoPro HERO9 Black à Prova D'águ...,Internacional
4,2022-05-21,https://www.amazon.com.br/C%C3%A2mera-GoPro-Fr...,AMAZON,EOS IMPORT,2540.73,10,254.10,2541.0,B08KXN6BNH,Câmera GoPro HERO9 Black à Prova D'águ...,Internacional
5,2022-05-21,https://www.amazon.com.br/C%C3%82MERA-GOPRO-BL...,AMAZON,EOS IMPORT,2599.00,10,259.90,2599.0,B08ZSVP9T7,CÂMERA GOPRO HERO 8 BLACK ESSENCIAL BU...,Internacional
6,2022-05-21,https://www.amazon.com.br/C%C3%A2mera-Digital-...,AMAZON,EOS IMPORT,3799.00,10,379.90,3799.0,B087ZP2GW4,Câmera Digital GoPro Max 360o - CHDHZ-...,Internacional
7,2022-05-21,https://www.amazon.com.br/Camera-DJI-Osmo-Acti...,AMAZON,HuaYe Tech,1982.90,10,198.29,1982.9,B07RJMK2GV,Camera DJI Osmo Action 4K Sensor CMOS ...,Internacional


In [40]:
Df_final.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Amazon_Motorola.xlsx", index=False)

In [21]:
Df_final.dtypes

DATE                    object
URL                     object
MARKETPLACE             object
SELLER                  object
PRICE                  float64
PARCEL                   int32
INSTALLMENT            float64
INSTALLMENT_PAYMENT    float64
ID                      object
PRODUCT                 object
INTERNACIONAL           object
MORE                     int32
dtype: object

In [22]:
Df_more_offers = Df_final[Df_final['MORE'] != 0]
Df_more_offers

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
3,2022-05-21,https://www.amazon.com.br/Gopro-Black-prova-D%...,AMAZON,EOS IMPORT,2299.00,10,229.90,2299.0,B07WSRXMS8,Gopro Hero 8 Black 4k à prova D'água C...,Internacional,4
7,2022-05-21,https://www.amazon.com.br/GoPro-Frontal-HyperS...,AMAZON,EOS IMPORT,3207.25,10,320.77,3207.7,B09D15SFMQ,Câmera GoPro HERO10 Black à Prova com ...,Internacional,5
13,2022-05-21,https://www.amazon.com.br/C%C3%A2mera-Cart%C3%...,AMAZON,EOS IMPORT,3000.00,10,300.00,3000.0,B08R2XCM9K,Câmera GoPro HERO9 Black à Prova D'águ...,Internacional,2
14,2022-05-21,https://www.amazon.com.br/C%C3%A2mera-GoPro-Fr...,AMAZON,EOS IMPORT,2540.73,10,254.10,2541.0,B08KXN6BNH,Câmera GoPro HERO9 Black à Prova D'águ...,Internacional,5
18,2022-05-21,https://www.amazon.com.br/C%C3%82MERA-GOPRO-BL...,AMAZON,EOS IMPORT,2599.00,10,259.90,2599.0,B08ZSVP9T7,CÂMERA GOPRO HERO 8 BLACK ESSENCIAL BU...,Internacional,5
42,2022-05-21,https://www.amazon.com.br/Camera-DJI-Osmo-Acti...,AMAZON,HuaYe Tech,1982.90,10,198.29,1982.9,B07RJMK2GV,Camera DJI Osmo Action 4K Sensor CMOS ...,Internacional,7


In [23]:
Amazon_ID_More = []

In [24]:
def search_more_offers(ASIN):
    global Amazon_seller_more
    
    time.sleep(5)

    new_url = "https://www.amazon.com.br/gp/product/ajax/?asin=" + ASIN + "&pageno=1&experienceId=aodAjaxMain"

    driver.get(new_url)
    
    time.sleep(5)

    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')

    Soup = BeautifulSoup(html_str, 'html.parser')

    for seller in Soup.find_all(class_='a-size-small a-link-normal')[4:]:
        Amazon_seller_more.append(seller.text)
        

    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Política de devolução' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Apagar tudo' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if len(s) > 1]

    for price in Soup.find_all(class_='a-offscreen')[2:]:
        Amazon_price_more.append(price.text)
        Amazon_ID_More.append(ASIN)

In [25]:
for url in tqdm(Df_more_offers['ID']):
    search_more_offers(url)

100%|██████████| 6/6 [01:05<00:00, 11.00s/it]


In [26]:
print(len(Amazon_seller_more))
print(len(Amazon_ID_More))
print(len(Amazon_price_more))

21
21
21


In [148]:
Dataframe_More = pd.DataFrame()
Dataframe_More['ID'] = Amazon_ID_More

Dataframe_More['DATE'] = pd.to_datetime('today', errors='ignore').date()

Dataframe_More['MARKETPLACE'] = 'AMAZON'

Dataframe_More['SELLER'] = Amazon_seller_more

Dataframe_More['PRICE'] = Amazon_price_more
Dataframe_More['PRICE'] = Dataframe_More['PRICE'].str.replace(".","", regex=True)
Dataframe_More['PRICE'] = Dataframe_More['PRICE'].str.replace("R$","", regex=False)
Dataframe_More['PRICE'] = Dataframe_More['PRICE'].str.replace(",",".", regex=True)
Dataframe_More['PRICE'] = Dataframe_More['PRICE'].astype('float')

Dataframe_More['PARCEL'] = 10

Dataframe_More['INSTALLMENT'] = Dataframe_More['PRICE'] / Dataframe_More['PARCEL']

Dataframe_More['INSTALLMENT_PAYMENT'] = Dataframe_More['PRICE'] * Dataframe_More['PARCEL']

Dataframe_More['INTERNACIONAL'] = 'ERRO'

url_names = []
for id in Dataframe_More['ID']:
    url_names.append(Df_final.loc[Df_final['ID'] == id,'URL'].values[0])

products_names = []
for id in Dataframe_More['ID']:
    products_names.append(Df_final.loc[Df_final['ID'] == id,'PRODUCT'].values[0])

Dataframe_More['URL'] = url_names
Dataframe_More['PRODUCT'] = products_names

Dataframe_More

,ID,DATE,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,INTERNACIONAL,URL,PRODUCT
0,B07WSRXMS8,2022-05-21,AMAZON,LIVE IMPORTS,2299.00,10,229.900,22990.0,ERRO,https://www.amazon.com.br/Gopro-Black-prova-D%...,Gopro Hero 8 Black 4k à prova D'água C...
1,B07WSRXMS8,2022-05-21,AMAZON,King Pro St,2289.00,10,228.900,22890.0,ERRO,https://www.amazon.com.br/Gopro-Black-prova-D%...,Gopro Hero 8 Black 4k à prova D'água C...
2,B07WSRXMS8,2022-05-21,AMAZON,GRUPO MAVERICK,2599.00,10,259.900,25990.0,ERRO,https://www.amazon.com.br/Gopro-Black-prova-D%...,Gopro Hero 8 Black 4k à prova D'água C...
3,B09D15SFMQ,2022-05-21,AMAZON,LIVE IMPORTS,3207.25,10,320.725,32072.5,ERRO,https://www.amazon.com.br/GoPro-Frontal-HyperS...,Câmera GoPro HERO10 Black à Prova com ...
4,B09D15SFMQ,2022-05-21,AMAZON,King Pro St,3219.00,10,321.900,32190.0,ERRO,https://www.amazon.com.br/GoPro-Frontal-HyperS...,Câmera GoPro HERO10 Black à Prova com ...
5,B09D15SFMQ,2022-05-21,AMAZON,GRUPO MAVERICK,4299.99,10,429.999,42999.9,ERRO,https://www.amazon.com.br/GoPro-Frontal-HyperS...,Câmera GoPro HERO10 Black à Prova com ...
6,B09D15SFMQ,2022-05-21,AMAZON,MeuDome,4799.00,10,479.900,47990.0,ERRO,https://www.amazon.com.br/GoPro-Frontal-HyperS...,Câmera GoPro HERO10 Black à Prova com ...
7,B08R2XCM9K,2022-05-21,AMAZON,EOS IMPORT,2999.00,10,299.900,29990.0,ERRO,https://www.amazon.com.br/C%C3%A2mera-Cart%C3%...,Câmera GoPro HERO9 Black à Prova D'águ...
8,B08KXN6BNH,2022-05-21,AMAZON,King Pro St,2599.00,10,259.900,25990.0,ERRO,https://www.amazon.com.br/C%C3%A2mera-GoPro-Fr...,Câmera GoPro HERO9 Black à Prova D'águ...
9,B08KXN6BNH,2022-05-21,AMAZON,King Pro St,2739.00,10,273.900,27390.0,ERRO,https://www.amazon.com.br/C%C3%A2mera-GoPro-Fr...,Câmera GoPro HERO9 Black à Prova D'águ...


In [130]:
url_names = []
for id in Dataframe_More['ID']:
    url_names.append(Df_final.loc[Df_final['ID'] == id,'URL'])

In [132]:
len(url_names)

21

In [114]:
Df_final.loc[Df_final['ID'] == Dataframe_More['ID'],'URL']

ValueError: Can only compare identically-labeled Series objects

In [149]:
Df_teste = pd.concat([Df_final, Dataframe_More])
Df_teste = Df_teste.sort_values(by='ID')

In [150]:
Df_teste

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL
20,2022-05-21,https://www.amazon.com.br/Camera-DJI-Osmo-Acti...,AMAZON,DigitalShoppStore,4100.00,10,410.000,41000.0,B07RJMK2GV,Camera DJI Osmo Action 4K Sensor CMOS ...,ERRO
18,2022-05-21,https://www.amazon.com.br/Camera-DJI-Osmo-Acti...,AMAZON,Bits & Bytes Tecnologia,2851.27,10,285.127,28512.7,B07RJMK2GV,Camera DJI Osmo Action 4K Sensor CMOS ...,ERRO
17,2022-05-21,https://www.amazon.com.br/Camera-DJI-Osmo-Acti...,AMAZON,BC INFO,2712.67,10,271.267,27126.7,B07RJMK2GV,Camera DJI Osmo Action 4K Sensor CMOS ...,ERRO
16,2022-05-21,https://www.amazon.com.br/Camera-DJI-Osmo-Acti...,AMAZON,Xalo Comercio de Produtos,2420.00,10,242.000,24200.0,B07RJMK2GV,Camera DJI Osmo Action 4K Sensor CMOS ...,ERRO
15,2022-05-21,https://www.amazon.com.br/Camera-DJI-Osmo-Acti...,AMAZON,QanHorse,1983.00,10,198.300,19830.0,B07RJMK2GV,Camera DJI Osmo Action 4K Sensor CMOS ...,ERRO
19,2022-05-21,https://www.amazon.com.br/Camera-DJI-Osmo-Acti...,AMAZON,Elefante Store,3605.23,10,360.523,36052.3,B07RJMK2GV,Camera DJI Osmo Action 4K Sensor CMOS ...,ERRO
7,2022-05-21,https://www.amazon.com.br/Camera-DJI-Osmo-Acti...,AMAZON,HuaYe Tech,1982.90,10,198.290,1982.9,B07RJMK2GV,Camera DJI Osmo Action 4K Sensor CMOS ...,Internacional
2,2022-05-21,https://www.amazon.com.br/Gopro-Black-prova-D%...,AMAZON,GRUPO MAVERICK,2599.00,10,259.900,25990.0,B07WSRXMS8,Gopro Hero 8 Black 4k à prova D'água C...,ERRO
1,2022-05-21,https://www.amazon.com.br/Gopro-Black-prova-D%...,AMAZON,King Pro St,2289.00,10,228.900,22890.0,B07WSRXMS8,Gopro Hero 8 Black 4k à prova D'água C...,ERRO
0,2022-05-21,https://www.amazon.com.br/Gopro-Black-prova-D%...,AMAZON,EOS IMPORT,2299.00,10,229.900,2299.0,B07WSRXMS8,Gopro Hero 8 Black 4k à prova D'água C...,Internacional


In [105]:
Dataframe_More['DATE'] = pd.to_datetime('today', errors='ignore').date()
#Dataframe_More['URL'] = [Df_final.loc[Df_final['ID'] == Dataframe_More['ID'], 'URL']]

ValueError: Can only compare identically-labeled Series objects

In [117]:
Dataframe_More

,ID,Seller,Price,DATE,URL
0,B07WSRXMS8,LIVE IMPORTS,"R$ 2.299,00",2022-05-21,https://www.amazon.com.br/Gopro-Black-prova-D%...
1,B07WSRXMS8,King Pro St,"R$ 2.289,00",2022-05-21,https://www.amazon.com.br/GoPro-Frontal-HyperS...
2,B07WSRXMS8,GRUPO MAVERICK,"R$ 2.599,00",2022-05-21,https://www.amazon.com.br/C%C3%A2mera-GoPro-HE...
3,B09D15SFMQ,LIVE IMPORTS,"R$ 3.207,25",2022-05-21,https://www.amazon.com.br/C%C3%A2mera-Cart%C3%...
4,B09D15SFMQ,King Pro St,"R$ 3.219,00",2022-05-21,https://www.amazon.com.br/C%C3%A2mera-GoPro-Fr...
5,B09D15SFMQ,GRUPO MAVERICK,"R$ 4.299,99",2022-05-21,https://www.amazon.com.br/C%C3%82MERA-GOPRO-BL...
6,B09D15SFMQ,MeuDome,"R$ 4.799,00",2022-05-21,https://www.amazon.com.br/C%C3%A2mera-Digital-...
7,B08R2XCM9K,EOS IMPORT,"R$ 2.999,00",2022-05-21,https://www.amazon.com.br/Camera-DJI-Osmo-Acti...
8,B08KXN6BNH,King Pro St,"R$ 2.599,00",2022-05-21,NaN
9,B08KXN6BNH,King Pro St,"R$ 2.739,00",2022-05-21,NaN


In [63]:
#Df_final = Df_final.drop(columns=['MORE'])
Df_final

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL
0,2022-05-21,https://www.amazon.com.br/Gopro-Black-prova-D%...,AMAZON,EOS IMPORT,2299.00,10,229.90,2299.0,B07WSRXMS8,Gopro Hero 8 Black 4k à prova D'água C...,Internacional
1,2022-05-21,https://www.amazon.com.br/GoPro-Frontal-HyperS...,AMAZON,EOS IMPORT,3207.25,10,320.77,3207.7,B09D15SFMQ,Câmera GoPro HERO10 Black à Prova com ...,Internacional
2,2022-05-21,https://www.amazon.com.br/C%C3%A2mera-GoPro-HE...,AMAZON,Amazon,4688.20,10,468.82,4688.2,B093JKGDJZ,Câmera GoPro HERO9 Black - Kit Trip Pr...,Internacional
3,2022-05-21,https://www.amazon.com.br/C%C3%A2mera-Cart%C3%...,AMAZON,EOS IMPORT,3000.00,10,300.00,3000.0,B08R2XCM9K,Câmera GoPro HERO9 Black à Prova D'águ...,Internacional
4,2022-05-21,https://www.amazon.com.br/C%C3%A2mera-GoPro-Fr...,AMAZON,EOS IMPORT,2540.73,10,254.10,2541.0,B08KXN6BNH,Câmera GoPro HERO9 Black à Prova D'águ...,Internacional
5,2022-05-21,https://www.amazon.com.br/C%C3%82MERA-GOPRO-BL...,AMAZON,EOS IMPORT,2599.00,10,259.90,2599.0,B08ZSVP9T7,CÂMERA GOPRO HERO 8 BLACK ESSENCIAL BU...,Internacional
6,2022-05-21,https://www.amazon.com.br/C%C3%A2mera-Digital-...,AMAZON,EOS IMPORT,3799.00,10,379.90,3799.0,B087ZP2GW4,Câmera Digital GoPro Max 360o - CHDHZ-...,Internacional
7,2022-05-21,https://www.amazon.com.br/Camera-DJI-Osmo-Acti...,AMAZON,HuaYe Tech,1982.90,10,198.29,1982.9,B07RJMK2GV,Camera DJI Osmo Action 4K Sensor CMOS ...,Internacional


In [38]:
Dataframe_More.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Amazon_More_Wacom.xlsx", index=False)

In [33]:
Amazon_seller_more =  [s for s in Amazon_seller_more if len(s) > 1]
print(len(Amazon_seller_more))

187


In [39]:
Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Política de devolução' in s]
Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Apagar tudo' in s]

print(len(Amazon_seller_more))

0


In [37]:
Amazon_seller_more

[' Tudo à Beça',
 'Política de devolução',
 ' Gigantec',
 'Política de devolução',
 ' INFOSHOP.',
 'Política de devolução',
 ' Thaun Shop',
 'Política de devolução',
 ' Tendinfo',
 'Política de devolução',
 ' Triplenet Pricing INC',
 'Política de devolução',
 '',
 'Apagar tudo',
 ' O MUNDO PARA VOCÊ',
 'Política de devolução',
 ' ZAPATA STORE SP',
 'Política de devolução',
 ' OFICINA DOS BITS',
 'Política de devolução',
 ' Syma Solutions',
 'Política de devolução',
 ' Conect Magazine',
 'Política de devolução',
 ' IBYTE',
 'Política de devolução',
 ' Triplenet Pricing INC',
 'Política de devolução',
 ' Triplenet Pricing INC',
 'Política de devolução',
 ' MAGIC WORLD ELETRONICOS',
 'Política de devolução',
 ' REX INFORMATICA',
 'Política de devolução',
 ' Tudo à Beça',
 'Política de devolução',
 ' W-TECH IMPORTS',
 'Política de devolução',
 ' Kadri Tecnologia',
 'Política de devolução',
 ' E-SPOT',
 'Política de devolução',
 ' Tendinfo',
 'Política de devolução',
 '',
 'Apagar tudo',
 '

In [33]:
Amazon_ID_More = []

In [ ]:
Amazon_seller_more = []
Amazon_price_more = []


In [ ]:
Amazon_seller_more

In [ ]:
Amazon_price_more

In [ ]:

new_url = "https://www.amazon.com.br/gp/product/ajax/?asin=" + "B09D15SFMQ" + "&pageno=1&experienceId=aodAjaxMain"

driver.get(new_url)

time.sleep(5)

body_el = driver.find_element(By.CSS_SELECTOR, 'body')
html_str = body_el.get_attribute('innerHTML')

Soup = BeautifulSoup(html_str, 'html.parser')

for seller in Soup.find_all(class_='a-size-small a-link-normal')[4:]:
    print(seller.text)

for price in Soup.find_all(class_='a-offscreen')[2:]:
    print(price.text)


In [ ]:
for url in Df_more_offers['ID']:
    time.sleep(3)

    search = "https://www.amazon.com.br/gp/product/ajax/?asin=" + url + "&pageno=1&experienceId=aodAjaxMain"

    driver.get(search)
    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')

    print(search)

    Soup = BeautifulSoup(html_str, 'html.parser')

    for seller in Soup.find_all(class_='a-size-small a-link-normal'):
        print(seller.text)

In [ ]:
#Teste gráfico 
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10,10))
plt.boxplot(x=Df_final['PRICE'], vert=False)
plt.show()

In [ ]:
df_products = Df_final[Df_final['PRICE'] > 800]

In [ ]:
df_products